In [ ]:
#after uploading the files from data folder comment this cell and execute all cells Runtime>Run all
from google.colab import files
uploaded = files.upload()

Saving nlp-domain-pipeline.joblib to nlp-domain-pipeline.joblib
Saving nlp-le-preprocessing.joblib to nlp-le-preprocessing.joblib
Saving nlp-task-pipeline.joblib to nlp-task-pipeline.joblib
Saving other-domain-pipeline.joblib to other-domain-pipeline.joblib
Saving other-le-preprocessing.joblib to other-le-preprocessing.joblib
Saving other-task-pipeline.joblib to other-task-pipeline.joblib
Saving vision-domain-pipeline.joblib to vision-domain-pipeline.joblib
Saving vision-le-preprocessing.joblib to vision-le-preprocessing.joblib
Saving vision-task-pipeline.joblib to vision-task-pipeline.joblib


In [ ]:
pip install github3.py

     |████████████████████████████████| 153kB 7.9MB/s 
     |████████████████████████████████| 81kB 5.9MB/s 
     |████████████████████████████████| 2.6MB 11.7MB/s 


In [ ]:
pip install gitpython

     |████████████████████████████████| 163kB 7.6MB/s 
     |████████████████████████████████| 71kB 5.2MB/s 


In [ ]:
pip install bibtexparser

     |████████████████████████████████| 51kB 3.6MB/s 
  Created wheel for bibtexparser: filename=bibtexparser-1.2.0-cp36-none-any.whl size=36711 sha256=f2f60f268e4d5b9bc9c8f9ad5776021fb787b18a2184cd32ef23b5c9c17ae365
  Stored in directory: /root/.cache/pip/wheels/b2/5a/e7/867bcbc3a81c15b675b931aa19b6698375c5a5e90419a366db
Successfully built bibtexparser


In [ ]:
pip install arxiv

     |████████████████████████████████| 81kB 3.7MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-cp36-none-any.whl size=6066 sha256=dbefbd1bc439639def4fde591cc7984d13a7651c3b61eb7043bbe043b565cc32
  Stored in directory: /root/.cache/pip/wheels/f1/80/5a/444ba08a550cdd241bd9baf8bae44be750efe370adb944506a
Successfully built sgmllib3k


In [ ]:
pip install Repo

  Created wheel for Repo: filename=repo-0.1.0-cp36-none-any.whl size=12573 sha256=e66b9bd68dbf7001e0f3e456c611d5248f2369b2032b863c8eac89d278d61a97
  Stored in directory: /root/.cache/pip/wheels/da/c6/66/173760638daf5d57b54c7ee130caf874d26055cc16eb832012
  Created wheel for configobj: filename=configobj-5.0.6-cp36-none-any.whl size=34545 sha256=34b986004bdb0a54f01ad24cd9aa696c057ffd27dd413a924a2995a95b9858fd
  Stored in directory: /root/.cache/pip/wheels/f1/e4/16/4981ca97c2d65106b49861e0b35e2660695be7219a2d351ee0
Successfully built Repo configobj


In [ ]:
import sys
import traceback

In [ ]:
import os
import fnmatch
import tempfile
import re
import json
from git import Repo

def extract_modules(text):
    importsRegEx = r"(?:import|from) ([\w\-]+)((?:(,\s*))[\w\-]+)*\s*(?:\s+as\s+\w*)?(?:import\s\w*)?[\n.]"
    # importsRegEx = r"(?:from|import)\s+(\w+)(?:\s+as\s+\w*)?[\n.]"
    mods = re.findall(importsRegEx, text)
    # return mods
    # if len(mods) > 0:
    #     print(text, " --- ", mods)
    modules = []
    for imp in mods:
        for i in imp:
            if len(i.strip()) > 0:
                modules.append(i.replace(",","").strip())
    # # print(modules)
    return modules

def get_py_modules(org, repo, repoPath=""):
    # TODO implement directory
    # TODO implement single file
    # create a temporary directory
    with tempfile.TemporaryDirectory() as directory:
        # if org == "tensorflow":
        #     return ["tensorflow"]
        imports = []

        Repo.clone_from("git://"+"github.com/"+org+"/"+repo, directory, depth=1, no_checkout=False, single_branch=True)

        if len(repoPath) > 0 and repoPath[0] != "/":
            repoPath = "/"+repoPath
        fileName = None
        pathParts = repoPath.split("/")
        if pathParts[-1].find(".") > 0:
            fileName = pathParts[-1]
            repoPath = "/".join(pathParts[:-1])
        # print("pathParts", pathParts)
        # print("repoPath", repoPath)
        # print("fileName", fileName)
        all_code_lines = []
        for path, subdirs, files in os.walk(directory+repoPath):
            innerPath = path[len(directory):]
            if innerPath.startswith("/."):
                continue
            if "init_net.pb" in files and "predict_net.pb" in files:
                if "caffe2" not in imports:
                    imports.append("caffe2")
            for name in files:
                if fileName is not None and name != fileName:
                    continue
                # print(path, innerPath, name)
                if fnmatch.fnmatch(name, "*.py"):
                    try:
                        with open(os.path.join(path, name), mode="r") as f:
                            lines = f.read()
                            all_code_lines += lines
                            result = extract_modules(lines)
                            for imp in result:
                                if imp not in imports:
                                    imports.append(imp)
                    except Exception as e:
                        print("Unable to read python file", os.path.join(path, name))
                if fnmatch.fnmatch(name, "*.ipynb"):
                    try:
                        with open(os.path.join(path, name), mode="r") as f:
                            code = json.load(f)
                            if 'cells' in code:
                                lines = []
                                for cell in code['cells']:
                                    if cell['cell_type'] == 'code':
                                        for line in cell['source']:
                                            lines.append(line)
                                all_code_lines += lines
                                result = extract_modules(" ".join(lines))
                                for imp in result:
                                    if imp not in imports:
                                        imports.append(imp)
                    except Exception as e:
                        print("Unable to parse ipynb file", os.path.join(path, name))
        #print("".join(all_code_lines[:10]))
        return sorted(imports)


In [ ]:
import json
import base64, re

In [ ]:
FRAMEWORKS = ["tensorflow", "scikit-learn", "scikit_learn", "sklearn", "keras", "theanos", "torch", "caffe", "caffe2", "nltk", "theano", "lasagne", "mxnet"]

def repo_framework(repo_object):

    frameworks = {}
    requirements = get_file_from_repo(repo_object, "requirements.txt")
    if requirements is None:
        requirements = get_file_from_repo(repo_object, "requirement.txt")
    if requirements is not None:
        modules = requirements.split("\n")
        for m in modules:
            s = re.split('==|>=|~=|>',m.lower())
            if s[0].strip() in FRAMEWORKS:
                fw = s[0].strip()
                frameworks[fw] = ""
                if len(s) > 1:
                    frameworks[fw] = s[1].strip()

    return frameworks

In [ ]:
import os
# import json

try:
    import importlib.resources as pkg_resources
except ImportError:
    # Try backported to PY<37 `importlib_resources`.
    import importlib_resources as pkg_resources

libraries = {}

fwArray = []
fwEnv = os.getenv("frameworks", None)
if fwEnv is not None:
    fwArray = fwEnv.split(",")
else:
    with open("frameworks.txt", "r") as fwFile:
        fwArray = fwFile.read().splitlines()

# print("Frameworks array:", ",".join(fwArray))
for f in fwArray:
    if ":" in f:
        temp = f.split(":")
        libraries[temp[0].strip().lower()] = temp[1].strip()
    else:
        libraries[f.strip().lower()] = f.strip()


In [ ]:
"""
    Utility functions for detecting datasets in a README file or similar
"""

import re
try:
    import importlib.resources as pkg_resources
except ImportError:
    # Try backported to PY<37 `importlib_resources`.
    import importlib_resources as pkg_resources

LINK_PATTERN = "\[([^\]]+?)\]\(([^\)]+?)\)"
ANCHOR_PATTERN = "^#\S+"
BOUNDRY_PATTERN = "(\s|\W)?"

DATASET_LIST_PATH = "sota-datasets.csv"

def detect_datasets_list(readme):
    dataset_list = load_dataset_list()
    list_datasets = find_dataset_from_list(readme, dataset_list)
    return list_datasets

def detect_datasets(readme, blob_link):
    dataset_list = load_dataset_list()
    datasets_found = []
    link_datasets = get_dataset_links(readme, blob_link)
    list_datasets = find_dataset_from_list(readme, dataset_list)
    datasets_found += link_datasets

    # resolve possible duplicates between the two, prioritize link datasets
    for list_dataset in list_datasets:
        skip = False
        for link_dataset in link_datasets:
            if list_dataset["name"].lower() in link_dataset["name"].lower():
                skip = True
                break
        if not skip:
            datasets_found.append(list_dataset)

    #print(datasets_found)
    return datasets_found

def load_dataset_list(dataset_list_path=DATASET_LIST_PATH):
    datasets = set()
    header = True
    with open(dataset_list_path,"r") as f:
        for line in f:
            datasets.add(line.strip())
    return datasets

def find_dataset_from_list(readme, dataset_list):
    datasets_found = []
    for dataset in dataset_list:
        boundry_re = re.compile("(\W|\A)" + dataset.lower() + "(\W|\Z)", re.M)
        #print(boundry_re)
        #print(boundry_re.search(readme.lower()))
        if boundry_re.search(readme.lower()):
            d = {
                "name": dataset
            }
            datasets_found.append(d)
    return datasets_found

def get_dataset_links(readme, blob_link):
    link_re = re.compile(LINK_PATTERN)

    # extract out links and look for "dataset or data set" and an actual link
    matches = link_re.findall(readme)
    datasets = []
    if (matches and len(matches) > 0):
        for m in matches:
            if "dataset" in m[0].lower() or "data set" in m[0].lower() or "corpus" in m[0].lower():
                # ignore if anchor
                anchor_re = re.compile(ANCHOR_PATTERN)
                anchors = anchor_re.search(m[1])
                dataset = {}
                if anchors:
                    continue
                if "http" in m[1]:
                    dataset["name"] = m[0].strip()
                    dataset["connection"] = {
                        "name": "url",
                        "source": {
                            "url": m[1].strip()
                        }
                    }
                else:
                    # local link to file in repo
                    local_link = blob_link + "/" + m[1]
                    dataset["name"] = m[0].strip()
                    dataset["connection"] = {
                        "name": "url",
                        "source": {
                            "url": local_link
                        }
                    }
                datasets.append(dataset)
    return datasets

if __name__ == '__main__':
    print(detect_datasets("blah blah [mnist dataset](mnist) \n WMT2016 English-Romanian", ""))


[{'name': 'mnist dataset', 'connection': {'name': 'url', 'source': {'url': '/mnist'}}}, {'name': 'WMT2016 English-Romanian'}]


In [ ]:
import arxiv
import re
from dateutil import parser

# http://arxiv.org/abs/1502.05698
# https://arxiv.org/pdf/1512.03385.pdf

ARXIV_ABS_PATTERN = "https?:\/\/arxiv.org\/abs\/(\d+\.\d+)"
ARXIV_PDF_PATTERN = "https?:\/\/arxiv.org\/pdf\/(\d+\.\d+)\.pdf"
ARXIV_ID_PATTERN = "arXiv:(\d+\.\d+)"

# Given url, returns None if not arxiv or id if is
def parse_arxiv_url(url):
    arxiv_abs_re = re.compile(ARXIV_ABS_PATTERN)
    arxiv_pdf_re = re.compile(ARXIV_PDF_PATTERN)

    found = arxiv_abs_re.search(url)
    if found:
        return found.group(1)
    found = arxiv_pdf_re.search(url)
    if found:
        return found.group(1)

    return None

# Given full readme, look for arxiv and return info about each paper
def look_for_arxiv_fulltext(text):
    arxiv_abs_re = re.compile(ARXIV_ABS_PATTERN)
    arxiv_pdf_re = re.compile(ARXIV_PDF_PATTERN)

    ids = set()
    found = arxiv_abs_re.findall(text)
    if found and len(found) > 0:
        for f in found:
            ids.add(f)

    found = arxiv_pdf_re.findall(text)
    if found and len(found) > 0:
        for f in found:
            ids.add(f)

    paper_info = []
    for i in ids:
        info = get_arxiv_id(i)
        if info:
            paper_info.append(info)
    return paper_info

# Given array of lines, look for arxiv and return info about each paper
def look_for_arxiv(lines, start=0, end=None):
    arxiv_abs_re = re.compile(ARXIV_ABS_PATTERN)
    arxiv_pdf_re = re.compile(ARXIV_PDF_PATTERN)

    if not end:
        end = len(lines)-1

    ids = set()
    for i in range(start,end+1):
        line = lines[i]
        #print(line)
        found = arxiv_abs_re.search(line)
        if found:
            ids.add(found.group(1))
        found = arxiv_pdf_re.search(line)
        if found:
            ids.add(found.group(1))

    paper_info = []
    for i in ids:
        info = get_arxiv_id(i)
        if info:
            paper_info.append(info)
    return paper_info

# Given text, look for arxiv and return info about paper
def look_for_arxiv_id(text):
    arxiv_re = re.compile(ARXIV_ID_PATTERN)
    match = arxiv_re.search(text)
    if match:
        arxiv_id = match.group(1)
        arxiv_info = get_arxiv_id(arxiv_id)
        return arxiv_info
    return None

def get_arxiv_id(id):
    results = arxiv.query(id_list=[id])
    paper_info = None
    # assume one result
    if len(results) > 0:
        result = results[0]
        published = parser.parse(result.published)
        paper_info = {
            "title": result.title,
            "authors": result.authors,
            "arxiv": id,
            "year": published.year,
            "url": result.arxiv_url
        }
        if "summary" in result:
            paper_info["abstract"] = result.summary
    return paper_info


In [ ]:
import re
import bibtexparser

REF_PATTERNS = {
    # IBM MAX 1, ex:* _S. Hershey, S. Chaudhuri, D. P. W. Ellis, J. F. Gemmeke, A. Jansen,\nR. C. Moore, M. Plakal, D. Platt, R. A. Saurous, B. Seybold et  al._,\n["CNN architectures for large-scale audio classification,"](https://arxiv.org/pdf/1609.09430.pdf) arXiv preprint\narXiv:1609.09430, 2016.
    '\* _([A-Za-z,\-\.\s]+?)_\s?,\s?\["?(.+?)"?\]\((\S+?)\)\s?arXiv preprint\s?(arXiv:\d+\.\d+), (\d+)\.': ["authors", "title", "url", "arxiv", "year"],
    # IBM MAX 2, ex:* _Qiuqiang Kong, Yong Xu, Wenwu Wang, Mark D. Plumbley_,["Audio Set classification with attention model: A probabilistic perspective."](https://arxiv.org/pdf/1711.00927.pdf) arXiv preprint arXiv:1711.00927 (2017).
    '\* _([A-Za-z,\-\.\s]+?)_\s?,\s?\["?(.+?)"?\]\((\S+?)\)\s?arXiv preprint\s?(arXiv:\d+\.\d+) \((\d+)\)\.': ["authors", "title", "url", "arxiv", "year"],
    # IBM MAX 3, ex:* _Jort F. Gemmeke, Daniel P. W. Ellis, Dylan Freedman, Aren Jansen, Wade Lawrence, R. Channing Moore, Manoj Plakal, Marvin Ritter_,["Audio set: An ontology and human-labeled dataset for audio events"](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/45857.pdf), IEEE ICASSP, 2017.
    '\* _([A-Za-z,\-\.\s]+?)_\s?,\s?\["?(.+?)"?\]\((\S+?)\)(?:\.|,) ([\s\S]+?), (\d+)\.': ["authors", "title", "url", "venue", "year"],
    # IBM MAX 4, ex:[1]<a name="ref1"></a> N. Shazeer, R. Doherty, C. Evans, C. Waterson., ["Swivel: Improving Embeddings\nby Noticing What's Missing"](https://arxiv.org/pdf/1602.02215.pdf) arXiv preprint arXiv:1602.02215 (2016)
    '\[\d+\]<a (?:.*)><\/a> ([A-Za-z,\-\.\s]+?)\s?\["?([\s\S]+?)"?\]\((\S+?)\)\s?arXiv preprint\s?(arXiv:\d+\.\d+) \((\d+)\)': ["authors", "title", "url", "arxiv", "year"],
    # IBM MAX 5, ex:[1] Jaderberg, Max, et al. ["Spatial Transformer Networks"](https://arxiv.org/pdf/1506.02025) arXiv preprint arXiv:1506.02025 (2015)
    '\[\d+\]\s?([A-Za-z,\-\.\s]+?)\s?\["?([\s\S]+?)"?\]\((\S+?)\)\s?arXiv preprint\s?(arXiv:\d+\.\d+) \((\d+)\)': ["authors", "title", "url", "arxiv", "year"],
    # IBM MAX 6, ex:* _D. Tran, L. Bourdev, R. Fergus, L. Torresani, M. Paluri_, [C3D: Generic Features for Video Analysis](http://vlg.cs.dartmouth.edu/c3d/)
    '\* _([A-Za-z,\-\.\s]+?)_\s?,\s?\["?(.+?)"?\]\((\S+?)\)': ["authors", "title", "url"],
    # IBM MAX 7, ex:* [Sports-1M Dataset Project Page](https://cs.stanford.edu/people/karpathy/deepvideo/)
    '\* \["?(.+?)"?\]\((\S+?)\)': ["title", "url"],
    # IBM MAX 8, ex: Rafal Jozefowicz, Oriol Vinyals, Mike Schuster, Noam Shazeer: “Exploring the Limits of Language Modeling”, 2016;\n[arXiv:1602.02410](http://arxiv.org/abs/1602.02410).
    '^([A-Za-z,\-\.\s]+?)\s?:\s?"?(.+?)"?, (\d+);\s?\["?(.+?)"?\]\((\S+?)\)\.': ["authors", "title", "year", "arxiv", "url"],
    # GitHub, ex:  - Raissi, Maziar, Paris Perdikaris, and George Em Karniadakis. "[Physics Informed Deep Learning (Part I): Data-driven Solutions of Nonlinear Partial Differential Equations](https://arxiv.org/abs/1711.10561)." arXiv preprint arXiv:1711.10561 (2017).
    '\s*?-\s?([A-Za-z,\-\.\s]+?).\s?"\["?(.+?)"?\]\((\S+?)\)."\s?arXiv preprint\s?(arXiv:\d+\.\d+)\s?\((\d+?)\)\.': ["authors", "title", "url", "arxiv", "year"],
}
URL_PATTERN = "^(?:http(s)?:\/\/)?[\w.-]+(?:\.[\w\.-]+)+[\w\-\._~:/?#[\]@!\$&'\(\)\*\+,;=.]+$"
URL_ENDER_EXCEPTIONS = (".json", ".yaml", ".py", ".ipynb")
ANCHOR_PATTERN = "^#\S+"
CODEBLOCK_PATTERN = "\s+?```(?:bibtex|)([\s\S]+?)```"

def make_paper_key(ref_info):
    title = ref_info["title"] if "title" in ref_info else ""
    authors = ""
    if "authors" in ref_info:
        if isinstance(ref_info["authors"], list):
            for a in ref_info["authors"]:
                authors += a
        elif isinstance(ref_info["authors"], str):
            authors = ref_info["authors"]
    return title + authors

def detect_references(readme):
    found = {}

    arxiv_ids = set()
    arxiv_papers = look_for_arxiv_fulltext(readme)

    for a in arxiv_papers:
        arxiv_ids.add(a["arxiv"])
        paper_key = make_paper_key(a)
        found[paper_key] = a

    bibtex_papers = codeblock_search(readme)
    for b in bibtex_papers:
        # skip if arxiv already found
        if "arxiv" in b:
            if b["arxiv"] in arxiv_ids:
                continue
            if "arxiv:" in b["arxiv"].lower():
                arxiv_id = b["arxiv"][b["arxiv"].lower().find("arxiv:")+len("arxiv:"):]
                if arxiv_id in arxiv_ids:
                    continue
        paper_key = make_paper_key(b)
        if paper_key not in found:
            found[paper_key] = b
        else:
            if len(b.keys()) > len(found[paper_key].keys()):
                found[paper_key] = b

    regex_papers = regex_search(readme)
    for r in regex_papers:
        # skip entire ref if arxiv already found
        if "arxiv" in r:
            if r["arxiv"] in arxiv_ids:
                continue
            if "arxiv:" in r["arxiv"].lower():
                arxiv_id = r["arxiv"][r["arxiv"].lower().find("arxiv:")+len("arxiv:"):]
                if arxiv_id in arxiv_ids:
                    continue
        if "url" in r:
            arxiv_id = parse_arxiv_url(r["url"])
            if arxiv_id and arxiv_id in arxiv_ids:
                continue
        paper_key = make_paper_key(r)
        # give priority to arxiv
        if paper_key not in found:
            found[paper_key] = r
        else:
            if len(r.keys()) > len(found[paper_key].keys()):
                found[paper_key] = r

    if len(found.values()) > 0:
        refs = []
        for ref in found.values():
            if "authors" in ref and isinstance(ref["authors"], str):
                ref["authors"] = ref["authors"].split(",")
            refs.append(ref)
        return {
            "references": refs
        }

    return {}

def regex_search(readme):
    ref_res = []
    for r in REF_PATTERNS.keys():
        ref_res.append(re.compile(r, re.M))

    found_refs = []
    for r in ref_res:
        result = r.findall(readme)
        headers = REF_PATTERNS[r.pattern]
        for match in result:
            ref = {}
            for i in range(len(headers)):
                header = headers[i]
                ref[header] = match[i]
            if "url" in ref:
                # skip if not actual URL
                url_re = re.compile(URL_PATTERN)
                urls = url_re.search(ref["url"])
                if not urls:
                    continue
                # skip if URL ends with certain strings
                if ref["url"].endswith(URL_ENDER_EXCEPTIONS):
                    continue
            # skip entire ref if url is an anchor (i.e. just #)
                anchor_re = re.compile(ANCHOR_PATTERN)
                anchors = anchor_re.search(ref["url"])
                if anchors:
                    continue

            found_refs.append(ref)
    return found_refs

def codeblock_search(readme):
    found_refs = []
    codeblock_re = re.compile(CODEBLOCK_PATTERN, re.M)
    result = codeblock_re.findall(readme)
    for match in result:
        #print("regex match", match)
        bib_results = bibtexparser.loads(match)
        if len(bib_results.entries) > 0:
            for e in bib_results.entries:
                ref = bibtexparser.customization.author(e)
                ref["authors"] = ref["author"]
                if "journal" in ref:
                    arxiv_info = look_for_arxiv_id(ref["journal"])
                    if arxiv_info:
                        # merge, give preference to whatever is longer
                        for k,v in arxiv_info.items():
                            if k == "arxiv":
                                ref[k] = v
                            try:
                                if k in ref:
                                    if len(v) > len(ref[k]):
                                        ref[k] = v
                                    else:
                                        ref[k] = v
                            except TypeError:
                                ref[k] = v
                found_refs.append(ref)

    return found_refs


In [ ]:
from github3 import login, GitHubEnterprise, exceptions

In [ ]:
def value_json_to_schema(schema_contents):
    try:
        value_info = json.loads(schema_contents)
    except json.decoder.JSONDecodeError as e:
        # This is to catch the case where value_info.json is stored in LFS
        print(e)
        return None
    #print(value_info)
    dimensions = None
    for k,v in value_info.items():
        if "data" in k:
            dimensions = v[1]
    input_schema = None
    if dimensions:
        input_schema = {
            "$schema": "http://json-schema.org/draft-04/schema#",
            "$id": "input_definition_data_schema.json",
            "id": "input_definition_data_schema.json",
            "title": "Input Data Schema from value_info.json",
            "type": "object",
            "required": ["input"],
            "properties": {
                "input": {
                  "description": "Images",
                  "dimensions": dimensions
                }
            }
        }
    return input_schema

In [ ]:
"""
    Utility functions used by the various parsers.
"""

from github3.exceptions import NotFoundError
import base64, json, yaml

BINARY_EXTS = (".checkpoint", "Dockerfile", ".caffemodel", ".pb", ".pbtxt",
    ".prototxt", ".ckpt", ".meta", ".index", ".onnx", ".joblib", ",pkl", ".h5",
    ".hdf5", "value_info.json")

# Expects repo object from github.repo, returns None if not found
def get_file_from_repo(repo_object, filepath):
    file_contents = None
    try:
        file_contents = repo_object.file_contents(filepath)
    except NotFoundError as e:
        return None
    if file_contents is not None:
        #contents = file_contents.decoded
        contents = file_contents.content
        if file_contents.encoding == "base64":
            contents = base64.b64decode(contents)
            contents = contents.decode('UTF-8')
        return contents

# slightly more complicated dict merger that is subobject aware
# for conflicts, dict2 takes priority
def merge_metadata(dict1, dict2):
    SUBOBJECTS = ["definition", "training", "trained_model", "provenance"]
    result = {**dict1, **dict2}
    for s in SUBOBJECTS:
        if s in dict1 and s in dict2:
            result[s] = {**dict1[s], **dict2[s]}
            # check for inner dicts
            for k, v in dict1[s].items():
                if k in dict2[s] and isinstance(v, dict):
                    result[s][k] = {**dict1[s][k], **dict2[s][k]}
    # special case of evaluations
    if "evaluations" in dict1 and "evaluations" in dict2:
        result["evaluations"] = dict1["evaluations"]
        for ev in dict2["evaluations"]:
            result["evaluations"].append(ev)
    # special note of authors: 2nd dict takes priority in list order
    if "authors" in dict2 and "authors" in dict1:
        result["authors"] = dict1["authors"]
        for a in dict2["authors"]:
            result["authors"].append(a)
    if "trained_model" in dict1 and "trained_model" in dict2:
        if "binaries" in dict1["trained_model"] and "binaries" in dict2["trained_model"]:
            result["trained_model"]["binaries"] = dict1["trained_model"]["binaries"] + dict2["trained_model"]["binaries"]
    return result

def get_blob_link(repo_object, tree_path=None, branch_name=None):
    if not branch_name:
        branch = repo_object.default_branch
    else:
        branch = branch_name
    url = repo_object.html_url
    if tree_path:
        folders = "/".join(tree_path)
        url += "/blob/" + branch + "/" + folders
    else:
        url += "/blob/" + branch
    return url

def get_tree_link(repo_object, tree_path=None, branch_name=None):
    if not branch_name:
        branch = repo_object.default_branch
    else:
        branch = branch_name
    url = repo_object.html_url
    if tree_path:
        folders = "/".join(tree_path)
        url += "/tree/" + branch + "/" + folders
    else:
        url += "/tree/" + branch
    return url

# returns all files in folder in tuples of (path, type, blob/tree)
# NOTE: only the first tree (at least for now, due to performance reasons)
def get_all_files_from_folder(repo_object, tree_path=None, exceptions=[]):
    if tree_path:
        folder_name = "/".join(tree_path)
    else:
        folder_name = "/"
    contents = repo_object.directory_contents(folder_name)
    file_contents = []
    for c in contents:
        try:
            if c[1].type == "dir" and c[0] not in exceptions:
                file_contents.append( (c[0], "dir", repo_object.tree(c[1].sha)) )
            else:
                file_contents.append( (c[0], "file", repo_object.blob(c[1].sha)) )
        except NotFoundError as e:
            # NOTE: seems to trigger for submodules, for now, go ahead and skip
            print(e)
            continue
    return file_contents

# given path and metadata_object, traverses object and inserts value into path
# to_object converts the value into an object
def path_to_object(metadata_object, value, path, to_object=False, is_yaml=False):
    path_list = path.split("/")
    # ensure each object before the last part of the path is created
    current_object = metadata_object
    for i in range(len(path_list)):
        path_part = path_list[i]

        if i < len(path_list) - 1:
            if path_part not in current_object:
                current_object[path_part] = {}
            current_object = current_object[path_part]
        else:
            # last part path
            if to_object:
                if is_yaml:
                    value = yaml.safe_load(value)
                else:
                    value = json.loads(value)
            current_object[path_part] = value
    return metadata_object

def is_binary_ext(path):
    if path.endswith(BINARY_EXTS):
        return True
    return False


In [ ]:
from os.path import join, dirname
from github3 import login, GitHubEnterprise, exceptions
import base64, re, os, json
from bs4 import BeautifulSoup
from markdown import markdown, Markdown
from io import StringIO

def markdown_to_text(markdown_string):
    """ Converts a markdown string to plaintext """
    # md -> html -> text since BeautifulSoup can extract text cleanly
    html = markdown(markdown_string)
    #print("html", html)
    # remove code snippets
    html = re.sub(r'<pre>(.*?)</pre>', ' ', html)
    html = re.sub(r'<code>(.*?)</code>', ' ', html)
    html = re.sub(r'<img>(.*?)</img>', ' ', html)
    html = re.sub(r'<a>(.*?)</a>', ' ', html)
    #print("html", html)
    # extract text
    soup = BeautifulSoup(html, "html.parser")
    # print("soup", soup)
    text = ''.join(soup.findAll(text=True))
    # print("text", text)
    return text

def unmark_element(element, stream=None):
    if stream is None:
        stream = StringIO()
    if element.text:
        stream.write(element.text)
    for sub in element:
        unmark_element(sub, stream)
    if element.tail:
        stream.write(element.tail)
    return stream.getvalue()

Markdown.output_formats["plain"] = unmark_element
__md = Markdown(output_format="plain")
__md.stripTopLevelTags = False

def unmark(text):
    return __md.convert(text)

def markdownToText(md):

    md = md.strip()
    # double clena up with 2 different approaches:
    # 1. html markdown
    text = markdown_to_text(md)
    # 2. by text element
    text = unmark(text)
    # remove duplicate symbols
    text = re.sub(r'(=|:|\t|\"|_|-){2,}', "", text)
    # remove empty lines
    # plain = re.sub(r'^\s*$', "", plain)
    text = re.sub(r'\n\s*\n', '\n', text, re.MULTILINE)
    # remove URL
    # text = re.sub(r'\(?https?://(?:[-\w.]|(?:%[\da-fA-F]{2}))+\)?', '', text)
    text = re.sub(r'\(?http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+\)?', '', text)
    #remove email
    text = re.sub(r'\S*@\S*\s?', '', text)

    # replace consecutives chars by a single one
    text = re.sub(r'(\s|\n)\1+', r'\1', text)

    return text.strip()

def readme_cleanup(markdown_readme):
    plain_readme = markdownToText(markdown_readme)

    return plain_readme


In [ ]:
"""
    Given a list of special files, looks for them and tries to parse them for metadata.
    Also supports yaml equivalents.

    Currently supported files:
        input_definition_data_schema.json / input_data_schema.json
        output_definition_data_schema.json / output_data_schema.json
        hyperparameter_schema.json
        hyperparameters.json
        input_trained_data_schema.json
        output_trained_data_schema.json
        features.json
        topology.json
        pipeline.json
"""

# from .gh_utils import get_file_from_repo, merge_metadata, path_to_object

SPECIAL_FILES = {
    "input_definition_data_schema": "definition/input_data_schema",
    "input_data_schema": "definition/input_data_schema",
    "output_definition_data_schema": "definition/output_data_schema",
    "output_data_schema": "definition/output_data_schema",
    "hyperparameter_schema": "definition/hyperparameter_schema",
    "topology": "definition/topology",
    "hyperparameters": "training/hyperparameters",
    "features": "training/features",
    "input_trained_data_schema": "trained_model/input_data_schema",
    "output_trained_data_schema": "trained_model/output_data_schema",
    "pipeline": "pipeline"
}

def is_special_file(filename):
    for special_file_names in SPECIAL_FILES.keys():
        if filename.endswith(special_file_names + ".json") or filename.endswith(special_file_names + ".yaml"):
            return True
    return False

def get_special_file(repo_object, filepath, propertypath, metadata, is_yaml=False):
    contents = get_file_from_repo(repo_object, filepath)
    if contents is None:
        return metadata
    return path_to_object(metadata, contents, propertypath, to_object=True, is_yaml=is_yaml)

def detect_special_files(repo_object, tree_path=None):
    result = {}

    for k, v in SPECIAL_FILES.items():
        if tree_path:
            folders = "/".join(tree_path)
            json_file = folders + "/" + k + ".json"
            yaml_file = folders + "/" + k + ".yaml"
        else:
            json_file = k + ".json"
            yaml_file = k + ".yaml"
        path = v
        result = get_special_file(repo_object, json_file, path, result)
        result = get_special_file(repo_object, yaml_file, path, result, is_yaml=True)

    return result


In [ ]:
from github3.exceptions import NotFoundError
import base64, re
LINK_PATTERN = "\[([^\]]+?)\]\(([^\)]+?)\)"
ANCHOR_PATTERN = "^#\S+"

MODEL_HEADER_PATTERN = "## Model(?:s| Metadata)\s+?([\s\S]+?)\s+?(?:#|\Z)"
MODEL_VALUE_HEADER_PATTERN = "##\s+?Model Value\n+?((?:###\s+?(?:[^#]+?)\n(?:^\|[\S\s]+?\|\n{2,}))*)"
TABLE_PATTERN = "###\s+?([^#]+?)\n(^\|[\S\s]+?\|\n{2,})"

MODEL_METRIC_SPECIAL_CASE = "Test Set"

SCHEMA_HEADER_PATTERN_PRE = "##\s*?"
SCHEMA_HEADER_PATTERN_POST = "\s+?```(?:json|)([\s\S]+?)```"

TABLE_HEADER_PATTERN_PRE = "##\s*?"
TABLE_HEADER_PATTERN_POST = "\s+?(^\|[\S\s]+?\|\n{2,})"

CODE_BLOCK_HEADERS_DICT = {
    "Input(?: Definition| ) Data Schema": "definition/input_data_schema",
    "Output(?: Definition| ) Data Schema": "definition/output_data_schema",
    "Hyperparameter Schema": "definition/hyperparameter_schema",
    "Input Trained Data Schema": "trained_model/input_data_schema",
    "Output Trained Data Schema": "trained_model/output_data_schema"
}

README_FILES = ["README.md", "README", "readme.md", "readme"]
#INPUT_TRAINED_SCHEMA_HEADER = "## Input Trained Data Schema\s+?```(?:json)([\s\S]+?)```\s+?(?:#|\Z)"

def is_readme_file(filename):
    for readme_file_names in README_FILES:
        if filename.endswith(readme_file_names):
            return True
    return False

def get_readme_contents(repo_object):
    try:
        readme = repo_object.readme()
    except NotFoundError as e:
        return None
    contents = readme.content
    if readme.encoding == "base64":
        contents = base64.b64decode(contents)
        contents = contents.decode('UTF-8')
    return contents

def get_readme_contents_from_sha(repo_object, sha):
    readme_blob = repo_object.blob(readme_sha)
    readme = readme_blob.decode_content()
    return readme

def get_readme_contents_from_path(repo_object, tree_path):
    full_path = ""
    for folder in tree_path:
        full_path += folder +"/"
    readme = None
    for readme_names in README_FILES:
        readme_path = full_path + readme_names
        readme = get_file_from_repo(repo_object, readme_path)
        if readme:
            break
    return readme

def get_readme_contents_from_docstring(repo_object, sha):
    code_blob = repo_object.blob(sha)
    code = code_blob.decode_content()
    # only read the docstring ('''....''')
    start = None
    end = None
    lines = code.split("\n")
    for i in range(len(lines)):
        line = lines[i]
        #print(line)
        if line.startswith("'''") or line.startswith('"""'):
            if start is None:
                start = i
            elif end is None:
                end = i
                break
    if start is None or end is None:
        return None
    docstring = ""
    for i in range(start,end+1):
        line = lines[i].strip()
        # remove start of docstring
        if i == start:
            docstring_start = line.find("'''")
            if docstring_start == -1:
                docstring_start = line.find('"""')
            line = line[docstring_start+3:]
        # remove end of docstring
        if i == end:
            docstring_end = line.find("'''")
            if docstring_end == -1:
                docstring_end = line.find('"""')
            line = line[:docstring_end]
        docstring += line + "\n"
    return docstring

def get_readme_title(readme):
    link_re = re.compile(LINK_PATTERN)

    # Use readme title as model name
    lines = readme.splitlines()
    line_index = 0
    title = None
    while line_index < len(lines):
        line = lines[line_index].strip()
        skip = False

        if line == "":
            skip = True

        # skip cases
        if line.lower() == "introduction" or line.lower().startswith("*"):
            skip = True

        # special cases
        if "travis-ci" in line.lower():
            skip = True

        # special RST cases
        if line.lower().startswith(".. ") or line.lower().startswith(":target:"):
            skip = True

        # another special RST case
        rst_table_re = re.compile("^(\|[^\|]+?\|_\s*)+")
        matches = rst_table_re.search(line)
        if matches:
            skip = True

        # skip if line is a link and image
        link_image_re = re.compile("^\[!\[([^\]]+?)\]\(([^\)]+?)\)\]\(([^\)]+?)\)")
        matches = link_image_re.search(line)
        if matches:
            skip = True

        # skip if line is an Image
        image_re = re.compile("^!\[([^\]]+?)\]\(([^\)]+?)\)")
        matches = image_re.search(line)
        if matches:
            skip = True

        # skip if line is nothing but "-" or "=" or "*" or "_"
        lines_re = re.compile("^[-=\*_]+")
        matches = lines_re.search(line)
        if matches:
            skip = True

        if not skip:
            title = line
            # remove hashes in front and/or back if they exist
            title_re = re.compile("^#+ ([^#]+)#*$")
            matches = title_re.search(title)
            if (matches):
                title = matches.group(1).strip()

            # remove <tags> if they exist
            tags_re = re.compile("<[\S\s]+?>")
            matches = tags_re.findall(title)
            if (matches and len(matches) > 0):
                for m in matches:
                    title = title.replace(m, "")

            # if a link is in the title, replace it
            links = link_re.search(title)
            if links:
                title = title.replace(links.group(0), links.group(1))

            break

        line_index += 1

    return title

def get_dataset_links(readme, blob_link):
    link_re = re.compile(LINK_PATTERN)

    # extract out links and look for "dataset or data set" and an actual link
    matches = link_re.findall(readme)
    datasets = {}
    if (matches and len(matches) > 0):
        for m in matches:
            if "dataset" in m[0].lower() or "data set" in m[0].lower() or "corpus" in m[0].lower():
                # ignore if anchor
                anchor_re = re.compile(ANCHOR_PATTERN)
                anchors = anchor_re.search(m[1])
                if anchors:
                    continue
                if "http" in m[1]:
                    datasets[m[0]] = m[1]
                else:
                    # local link to file in repo
                    local_link = blob_link + "/" + m[1]
                    datasets[m[0]] = local_link
    return datasets

def check_codeblock(readme, header, metadata_path, metadata, to_object=False):
    header_pattern = SCHEMA_HEADER_PATTERN_PRE + header + SCHEMA_HEADER_PATTERN_POST
    header_re = re.compile(header_pattern, re.M | re.I)

    result = header_re.search(readme)
    if (result and len(result.groups()) > 0):
        code_block = result.group(1).strip()
        block_result = path_to_object({}, code_block, metadata_path, to_object)
        return merge_metadata(metadata, block_result)

    return metadata

def check_table(readme, header, metadata_path, metadata, header_value_dict=None):
    header_pattern = TABLE_HEADER_PATTERN_PRE + header + TABLE_HEADER_PATTERN_POST
    header_re = re.compile(header_pattern, re.M | re.I)

    result = header_re.search(readme)
    if (result and len(result.groups()) > 0):
        table_block = result.group(1).strip()
        table_result = parse_markdown_table(table_block, header_value_dict)
        if len(table_result) > 0:
            table_result = path_to_object(metadata, table_result, metadata_path)
        return table_result

    return metadata

# returns array of objects, optionally can define paths
def parse_markdown_table(metric_table_md, header_value_dict=None):
    rows = metric_table_md.split("\n")
    headers = rows[0].split("|")
    metadata_objects = []
    for r_index in range(2, len(rows)):
        metadata = None
        values = rows[r_index].split("|")
        for i in range(len(headers)):
            h = headers[i].strip()
            v = values[i].strip()

            if len(h) > 0:
                if not metadata:
                    metadata = {}
                if header_value_dict:
                    if h.lower() in header_value_dict:
                        metadata = path_to_object(metadata, v, header_value_dict[h.lower()])
                else:
                    metadata[h.lower()] = v

        if metadata:
            metadata_objects.append(metadata)

    return metadata_objects

def check_model_value_table(readme):
    metadata = {}
    header_re = re.compile(MODEL_VALUE_HEADER_PATTERN, re.M | re.I)
    result = header_re.search(readme)
    evaluations = []
    if (result and len(result.groups()) > 0):
        # add newlines for the regex pattern matching
        raw_table = result.group(1).strip() + "\n\n"
        table_re = re.compile(TABLE_PATTERN, re.M | re.I)
        results = table_re.findall(raw_table)
        metric_metadata = {
            "evaluation_type": "training_evaluation",
            "method": MODEL_METRIC_SPECIAL_CASE
        }
        metrics = []
        for r in results:
            metric_name = r[0]
            table_md = r[1].strip()
            parse_result = {
                "metric": metric_name,
            }
            # special case
            if metric_name.lower() == "cost of training":
                parse_result = parse_markdown_table(table_md)
                if len(parse_result) > 0:
                    metadata["training"] = {
                        "training_job": {
                            "costs": parse_result
                        }
                    }
            else:
                parse_results = parse_markdown_table(table_md)
                comparisons = []
                for pr in parse_results:
                    # special case that defines what the metric is
                    if "above" in pr and pr["above"].lower() == MODEL_METRIC_SPECIAL_CASE.lower():
                        parse_result["value"] = pr["measurement"]
                    else:
                        comparisons.append(pr)
                if len(comparisons) > 0:
                    parse_result["comparisons"] = comparisons
                metrics.append(parse_result)
        if len(metrics) > 0:
            metric_metadata["metrics"] = metrics

        evaluations.append(metric_metadata)
    if len(evaluations) > 0:
        metadata["evaluations"] = evaluations
    return metadata

def check_model_metadata_table(readme):
    metadata = { "domain": {} }
    header_re = re.compile(MODEL_HEADER_PATTERN, re.M | re.I)
    result = header_re.search(readme)
    if (result and len(result.groups()) > 0):
        raw_table = result.group(1).strip()
        rows = raw_table.split("\n")
        headers = rows[0].split("|")
        values = rows[2].split("|")
        for i in range(len(headers)):
            h = headers[i].strip()
            v = values[i].strip()

            if h == "Domain":
                metadata["domain"]["domain_type"] = v
            elif h == "Application":
                metadata["domain"]["tasks"] = [v]
            elif h == "Industry":
                metadata["domain"]["industries"] = [v]
            elif h == "Training Data" or h == "Datasets":
                # check if it's a link
                link_re = re.compile(LINK_PATTERN)
                matches = link_re.findall(v)
                datasets = []
                if (matches and len(matches) > 0):
                    for m in matches:
                        datasets.append({
                            "name": m[0],
                            "url": m[1]
                        })
                else:
                    datasets.append({
                        "name": v
                    })

                metadata["training"] = {
                    "datasets": datasets
                }
            elif h == "Input Data Format":
                if "definition" not in metadata:
                    metadata["definition"] = {}
                if "input_data_schema" not in metadata:
                    metadata["definition"]["input_data_schema"] = {}
                metadata["definition"]["input_data_schema"]["description"] = v
            elif h == "Framework":
                if "definition" not in metadata:
                    metadata["definition"] = {}
                metadata["definition"]["framework"] = v

    if len(metadata["domain"].keys()) == 0:
        del metadata["domain"]

    return metadata

def readme_parse_text(readme, blob_link, check_datasets=False):
    refs = detect_references(readme)
    metadata = check_model_metadata_table(readme)
    #print(refs)
    result = merge_metadata(refs, metadata)

    metadata = check_model_value_table(readme)
    result = merge_metadata(result, metadata)

    author_header_dict = {
        "name": "name",
        "email": "email",
        "Github Profile": "github_id",
        "organization": "organization"
    }
    result = check_table(readme, "Contributors", "authors", result, author_header_dict)

    # check for special codeblocks
    for header, path in CODE_BLOCK_HEADERS_DICT.items():
        result = check_codeblock(readme, header, path, result, to_object=True)

    # special case of a codeblock containing published docker image address
    docker_result = check_codeblock(readme, "Published Docker Image:", "trained_model/binaries", {})
    if "trained_model" in docker_result and "binaries" in docker_result["trained_model"]:
        docker_address = docker_result["trained_model"]["binaries"]
        docker_result["trained_model"]["binaries"] = [{
            "name": "Published Docker Image",
            "type": "docker",
            "description": "URL to a docker repository where docker image is published",
            "connection": {
                "name": "docker_url",
                "source": {
                    "url": docker_address
                }
            }
        }]
        result = merge_metadata(result, docker_result)

    title = get_readme_title(readme)
    if title:
        result["name"] = title

    if check_datasets:
        datasets = detect_datasets(readme, blob_link)
        if len(datasets) > 0:
            if "training" not in result:
                result["training"] = {
                    "datasets": []
                }
            if "datasets" not in result["training"]:
                result["training"]["datasets"] = []
            for d in datasets:
                result["training"]["datasets"].append(d)

    return result


def readme_parse(repo_object, branch_name=None, tree_path=None, check_datasets=False):
    result = {}
    blob_link = get_blob_link(repo_object, tree_path=tree_path, branch_name=branch_name)
    if not tree_path:
        readme = get_readme_contents(repo_object)
    else:
        readme = get_readme_contents_from_path(repo_object, tree_path)

    if not readme:
        return result

    return readme_parse_text(readme, blob_link, check_datasets)


In [ ]:
import os
# import json

try:
    import importlib.resources as pkg_resources
except ImportError:
    # Try backported to PY<37 `importlib_resources`.
    import importlib_resources as pkg_resources

libraries = {}

fwArray = []
fwEnv = os.getenv("frameworks", None)
if fwEnv is not None:
    fwArray = fwEnv.split(",")
else:
    with open("frameworks.txt","r") as fwFile:
        fwArray = fwFile.read().splitlines()

# print("Frameworks array:", ",".join(fwArray))
for f in fwArray:
    if ":" in f:
        temp = f.split(":")
        libraries[temp[0].strip().lower()] = temp[1].strip()
    else:
        libraries[f.strip().lower()] = f.strip()

# print("Frameworks libraries:", json.dumps(libraries, indent=2))


def getFrameworks(modules):
    fws = []
    for mod in modules:
        if mod.lower() in libraries.keys():
            fws.append(libraries[mod.lower()])
    return fws


In [ ]:
def extract_framework(repo_url):

    url_parts = repo_url.split("/")
    for i in range(0, len(url_parts)):
        url_parts[i] = url_parts[i].lower()

    #print(url_parts)
    org = url_parts[3]
    reponame = url_parts[4]
    repopath = "/".join(url_parts[7:len(url_parts)])
    if not repopath:
        repopath = ""

    response_json = {'success': False}

    response_json['success'] = True
    #response_json["modules"] = get_py_modules(org, reponame)
    modules = get_py_modules(org, reponame)
    response_json["frameworks"] = getFrameworks(modules)
    #print(response_json)
    return response_json

In [ ]:
from joblib import load
import numpy as np

# from . import models

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

try:
    import importlib.resources as pkg_resources
except ImportError:
    # Try backported to PY<37 `importlib_resources`.
    import importlib_resources as pkg_resources

vision_domain_pipeline = load("vision-domain-pipeline.joblib")

nlp_domain_pipeline = load("nlp-domain-pipeline.joblib")

other_domain_pipeline = load("other-domain-pipeline.joblib")
vision_task_pipeline = load("vision-task-pipeline.joblib")

vision_le = load("vision-le-preprocessing.joblib")

nlp_task_pipeline = load("nlp-task-pipeline.joblib")

nlp_le = load("nlp-le-preprocessing.joblib")

other_pipeline = load("other-task-pipeline.joblib")

other_le = load("other-le-preprocessing.joblib")

# vision_domain_pipeline = load('models/vision-domain-pipeline.joblib')
# nlp_domain_pipeline = load('models/nlp-domain-pipeline.joblib')
# other_domain_pipeline = load('models/other-domain-pipeline.joblib')
# vision_task_pipeline = load('models/vision-task-pipeline.joblib')
# vision_le = load('models/vision-le-preprocessing.joblib')
# nlp_task_pipeline = load('models/nlp-task-pipeline.joblib')
# nlp_le = load('models/nlp-le-preprocessing.joblib')
# other_pipeline = load('models/other-task-pipeline.joblib')
# other_le = load('models/other-le-preprocessing.joblib')


def domain_inference(readme):
    return_json = { "domain_type": "Unknown" }

    predicted_vision = vision_domain_pipeline.predict_proba([readme])
    predicted_nlp = nlp_domain_pipeline.predict_proba([readme])
    predicted_other = other_domain_pipeline.predict_proba([readme])

    categories = ["Computer Vision", "Natural Language Processing", "Other", "Unknown"]
    # print(predicted_vision, predicted_nlp, predicted_other)
    probs = np.array([ max(predicted_vision[0]), max(predicted_nlp[0]), max(predicted_other[0])])
    results = np.array([ np.argmax(predicted_vision[0]), np.argmax(predicted_nlp[0]), np.argmax(predicted_other[0])])
    # print(probs, results)

    # if only one is True
    cat_index = -1
    trues = np.sum(results)
    if trues == 0:
        cat_index = 3
    elif trues == 1:
        cat_index = np.where(results == 1)[0][0]
    elif trues >= 2:
        true_index = np.where(results == 1)[0]
        for i in range(len(results)):
            if i not in true_index:
                probs[i] = 0
        cat_index = np.argmax(probs)

    if cat_index != 3 and probs[cat_index] > 0.5:
        return_json["domain_type"] = categories[cat_index]
        return_json["domain_prob"] = probs[cat_index]
        # do the task or other
        if categories[cat_index] == "Computer Vision":
            predicted = vision_task_pipeline.predict_proba([readme])
            # print("predicted", predicted)
            prob = max(predicted[0])
            result_index = np.argmax(predicted[0])
            label = vision_le.inverse_transform([result_index])
            return_json["task"] = label[0]
            return_json["task_prob"] = prob
        elif categories[cat_index] == "Natural Language Processing":
            predicted = nlp_task_pipeline.predict_proba([readme])
            prob = max(predicted[0])
            result_index = np.argmax(predicted[0])
            label = nlp_le.inverse_transform([result_index])
            return_json["task"] = label[0]
            return_json["task_prob"] = prob
        else:
            predicted = other_pipeline.predict_proba([readme])
            prob = max(predicted[0])
            result_index = np.argmax(predicted[0])
            label = other_le.inverse_transform([result_index])
            return_json["domain_type"] = label[0]
            return_json["domain_prob"] = prob

    return return_json


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.svm.classes module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.svm. Anything that cannot be imported from sklearn.svm is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.label module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)


In [ ]:
class AIMMX:
    """ Client for AIMMX."""

    def __init__(self, public_gh_token, enterprise_gh_creds=None):
        """

        """
        self._public_token = public_gh_token
        self._gh = login(token=public_gh_token)
        if enterprise_gh_creds:
            self._enterprise_login = enterprise_gh_creds[0]
            self._enterprise_token = enterprise_gh_creds[1]
            gh = GitHubEnterprise("https://github.com")
            # self._gh_ent = gh.login(enterprise_gh_creds[0], password=enterprise_gh_creds[1])

    def repo_parse(self, repo_url):

        result = {}

        if "github.ibm.com" in repo_url:
            if not self._gh_ent:
                raise Exception("Must provide enterprise GitHub credentials to extract from enterprise repositories.")
            result["visibility"] = {
                "visibility": "private"
            }
            gh = self._gh_ent
        else:
            result["visibility"] = {
                "visibility": "public"
            }
            gh = self._gh

        s = repo_url.split("/")
        owner = s[3]
        repo_name = s[4]
        repo = gh.repository(owner, repo_name)

        print("Extracting AI model metadata from: ", repo)

        # check if the url given is a directory or single file
        branch = None
        tree_path = None
        blob_path = None
        if len(s) >= 8:
            branch = s[6]
            if s[5] == "tree":
                tree_path = s[7:]
            if s[5] == "blob":
                blob_path = s[7:]

        # NOTE: need preview accept header for topics, may be brittle
        gh.session.headers["Accept"] = "application/vnd.github.mercy-preview+json"

        result["name"] = repo.name
        result["definition"] = {
            "code": [{
                "type": "repo",
                "repo_type": "github",
                "owner": owner,
                "owner_type": repo.owner.as_dict()["type"],
                "name": repo_name,
                "url": repo_url,
                "stars": repo.stargazers_count,
                "pushed_at": str(repo.pushed_at),
                "created_at": str(repo.created_at),
                "language": repo.language
            }]
        }


        # Get all the files in the repo
        if not blob_path:
            files = get_all_files_from_folder(repo, tree_path=tree_path)
        else:
            # Single file case
            f = (blob_path[-1], "file", repo.file_contents("/".join(blob_path)))
            files = [f]
            tree_path = blob_path[:-1]
        blob_link = get_blob_link(repo, tree_path=tree_path, branch_name=branch)

        for f in files:
            if is_special_file(f[0]):
                continue
            if is_readme_file(f[0]):
                continue
            code_meta = {
                "type": "code",
                "name": f[0],
                "sha": f[2].sha,
                "filetype": f[1],
                "connection": {
                    "name": "github_url",
                    "source": {}
                }
            }
            if f[1] == "file":
                code_meta["size"] = f[2].size
                code_meta["connection"]["source"] = {"url": blob_link + "/" + f[0]}
            elif f[1] == "dir":
                code_meta["num_files"] = len(f[2].tree)
                code_meta["connection"]["source"] = {
                    "url": get_tree_link(repo, tree_path=tree_path, branch_name=branch) + "/" + f[0]
                }

            if is_binary_ext(f[0]):
                if "trained_model" not in result:
                    result["trained_model"] = {}
                    if "binaries" not in result["trained_model"]:
                        result["trained_model"]["binaries"] = []

                code_meta["type"] = "binary"
                result["trained_model"]["binaries"].append(code_meta)
            else:
                result["definition"]["code"].append(code_meta)

            # Caffe2 special case
            # NOTE: should later be refactored into a caffe2 specific portion
            if f[0] == "value_info.json":
                schema_contents = f[2].decoded
                input_schema = value_json_to_schema(schema_contents)
                if input_schema:
                    result["trained_model"]["input_data_schema"] = input_schema

        for c in repo.contributors():
            if "authors" not in result:
                result["authors"] = []
            user = gh.user(c.login)
            author = {}
            if (user.name and len(user.name) > 0):
                author["name"] = user.name
            else:
                author["name"] = c.login
            if (user.email and len(user.email) > 0):
                author["email"] = user.email
            author["github_id"] = c.login
            result["authors"].append(author)

        # Gets all the topics/tags in GitHub repo
        topics = repo.topics()
        if topics:
            result["tags"] = []
            for t in topics.names:
                result["tags"].append(t)

        repo_desc = None
        if repo.description:
            result["description"] = repo.description
            result["definition"]["code"][0]["description"] = repo.description
            repo_desc = repo.description

        extraction = {
            "type": "github",
            "url": repo_url,
            "stars": repo.stargazers_count,
            "issues": repo.has_issues
        }

        # Gets readme and does analysis
        readme_content = None
        # For single file case, treats docstring as the readme
        if blob_path:
            readme_content = get_readme_contents_from_docstring(repo, files[0][2].sha)
            if readme_content is not None:
                extraction["readme"] = readme_content
                extraction["readme_url"] = repo_url
        # Subfolder case and also if single file does not contain docstring
        if tree_path and (not blob_path or readme_content is None):
            readme_content = get_readme_contents_from_path(repo, tree_path)
            if readme_content is not None:
                extraction["readme"] = readme_content
                extraction["readme_url"] = repo_url
        # if subfolder case does not find a README, try with repo-level README
        if readme_content is None:
            readme_content = get_readme_contents(repo)
            if readme_content is not None:
                extraction["readme"] = readme_content
                extraction["readme_url"] = "/".join(s[:5])

        result["extraction"] = [extraction]

        repo_license = None
        try:
            if repo.license():
                repo_license = repo.license().license.name
                result["visibility"]["license"] = repo_license
                result["definition"]["code"][0]["license"] = repo_license
        except exceptions.NotFoundError:
            pass

        # outdated framework extraction
        # frameworks = repo_framework(repo)
        # if (frameworks and len(frameworks.keys()) > 0):
        #     result["definition"]["code"][0]["framework"] = []
        #     for fw,version in frameworks.items():
        #         fw_insert = {
        #             "name": fw
        #         }
        #         if version != "":
        #             fw_insert["version"] = version
        #         # NOTE: for now, framework is singular, change this later
        #         result["definition"]["code"][0]["framework"] = fw_insert

        readme_info = readme_parse(repo, branch_name=branch, check_datasets=True)
        readme_name = None
        if "name" in readme_info:
            readme_name = readme_info["name"]
        result = merge_metadata(result, readme_info)

        if tree_path:
            readme_info = readme_parse(repo, branch_name=branch, tree_path=tree_path, check_datasets=True)
            # special case for subfolders, if there's no other name, take it from the folder
            if "name" not in readme_info:
                readme_info["name"] = "/".join(tree_path)
            result = merge_metadata(result, readme_info)

        if blob_path:
            readme_info = readme_parse_text(readme_content, blob_link, check_datasets=True)
            result = merge_metadata(result, readme_info)

        # special case to add frameworks to code files
        if "definition" in result and "framework" in result["definition"]:
            if "code" in result["definition"]:
                result["definition"]["code"][0]["framework"] = {
                    "name": result["definition"].pop("framework")
                }

        # Special files check, doesn't make sense in single-file case
        if not blob_path:
            specialfiles = detect_special_files(repo, tree_path=tree_path)
            result = merge_metadata(result, specialfiles)

        # if README exists, attempt domain Inference and dataset detection
        if readme_content:
            plain_readme = readme_cleanup(readme_content)

            abstracts = ""
            if "references" in result:
                for r in result["references"]:
                    if "abstract" in r:
                        abstract = r["abstract"]
                        abstract = abstract.strip().replace("\n", " ")
                        abstracts +=  "\n{}".format(abstract)
            if len(abstracts) > 0:
                plain_readme += abstracts

            domain = domain_inference(plain_readme)
            result["domain"] = domain

            # NOTE: currently only running on abstracts, should refactor to run on abstract + readme
            abs_datasets = detect_datasets_list(abstracts)
            if len(abs_datasets) > 0:

                if "training" in result and "datasets" in result["training"]:
                    dataset_names = set()
                    datasets_to_add = []
                    for d in result["training"]["datasets"]:
                        dataset_names.add(d["name"].lower())
                    for d in abs_datasets:
                        if d["name"].lower() not in dataset_names:
                            datasets_to_add.append(d)
                    result["training"]["datasets"] += datasets_to_add
                else:
                    result["training"] = { "datasets": abs_datasets }

        # attempt to extract framework via cloning
        framework_result = extract_framework(repo_url)
        if framework_result["success"]:
            if "frameworks" in framework_result:
                result["extraction"][0]["frameworks"] = framework_result["frameworks"]
                if "code" in result["definition"]:
                    result["definition"]["code"][0]["frameworks"] = framework_result["frameworks"]
            if "modules" in framework_result:
                result["extraction"][0]["modules"] = framework_result["modules"]
                if "code" in result["definition"]:
                    result["definition"]["code"][0]["modules"] = framework_result["modules"]

        return result

Examples

In [ ]:
# To generate a github api key

#On GitHub, click your profile picture in the top-right

#Settings > Developer Settings > Personal access tokens > "Generate new token"


a_aimmx = AIMMX(public_gh_token="8166550b79e39b73676cbe1f255b705e36fb5f0e", enterprise_gh_creds=None)


In [ ]:
#example github links

# https://github.com/strawlab/braincode
# https://github.com/tensorlayer/srgan
# https://github.com/TachibanaYoshino/AnimeGAN
# https://github.com/YunjaeChoi/vaemols
# https://github.com/CR-Gjx/LeakGAN

import pprint
import git
metadata = a_aimmx.repo_parse(repo_url="https://github.com/kweonwooj/papers")
#metadata can also be downloaded as json file
# metadata = aimmx.repo_parse(repo)
#     with open(metadata["name"] + ".json", "w") as f:
#         json.dump(metadata, f, indent=4)
pprint.pprint(metadata, indent=4)

Extracting AI model metadata from:  kweonwooj/papers
{   'authors': [   {   'email': 'kweonwooj@gmail.com',
                       'github_id': 'kweonwooj',
                       'name': 'Kweon Woo Jung(Chris Jung)'}],
    'definition': {   'code': [   {   'created_at': '2017-09-27 02:30:43+00:00',
                                      'description': 'summary of ML papers '
                                                     "I've read",
                                      'frameworks': [],
                                      'language': None,
                                      'name': 'papers',
                                      'owner': 'kweonwooj',
                                      'owner_type': 'User',
                                      'pushed_at': '2018-07-27 07:30:07+00:00',
                                      'repo_type': 'github',
                                      'stars': 281,
                                      'type': 'repo',
                     